In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress

import sys
sys.path.append('..')
from Scripts.utils import (add_underscore_before_capital_letters)

# show all columns 
pd.set_option('display.max_columns', None)
# show entire rows
pd.set_option('display.max_rows', None)
# show all values in a cell
pd.set_option('display.max_colwidth', None)

In [2]:
# Carregamento e exploração dos dados

raw_df = pd.read_excel('../Data/raw_data.xlsx')
raw_df.head()

,codigo,nome,cidade,estado,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento,potencial
0,3304557060,Abolição,Rio de Janeiro,RJ,11676,1027,483,688,800,1675,2300,1784,2919,0,145,715,1242,1093,758,92,304,2501,932515.0,Médio
1,3304557138,Acari,Rio de Janeiro,RJ,27564,5131,2188,2697,2630,4810,5308,2403,2397,0,0,82,506,2040,2490,827,2506,931,588833.0,Baixo
2,3304557057,Água Santa,Rio de Janeiro,RJ,9003,883,399,597,762,1755,2076,1112,1419,0,96,404,652,644,522,77,254,2391,874200.0,Baixo
3,3304557031,Alto Da Boa Vista,Rio de Janeiro,RJ,9606,1072,538,660,685,1317,2007,1341,1986,114,178,393,517,945,584,137,286,3727,912226.0,Médio
4,3304557125,Anchieta,Rio de Janeiro,RJ,57222,7677,3774,4892,4600,8660,12272,7157,8190,0,0,1089,2821,5110,5422,1073,3261,1380,553020.0,Médio


In [3]:
# Verificando o tamanho do dataset
print('O dataset possui {} linhas e {} colunas'.format(raw_df.shape[0], raw_df.shape[1]))
print('-----------------------------------------')
# Verificando os tipos de dados
print('Tipos de dados:')
print(raw_df.dtypes)

O dataset possui 456 linhas e 24 colunas
-----------------------------------------
Tipos de dados:
codigo            int64
nome             object
cidade           object
estado           object
população         int64
popAte9           int64
popDe10a14        int64
popDe15a19        int64
popDe20a24        int64
popDe25a34        int64
popDe35a49        int64
popDe50a59        int64
popMaisDe60       int64
domiciliosA1      int64
domiciliosA2      int64
domiciliosB1      int64
domiciliosB2      int64
domiciliosC1      int64
domiciliosC2      int64
domiciliosD       int64
domiciliosE       int64
rendaMedia       object
faturamento     float64
potencial        object
dtype: object


In [4]:
print('-----------------------------------------')
print('Valores não numéricos na coluna rendaMedia:')
non_numeric_values = raw_df['rendaMedia'].notna() & raw_df['rendaMedia'].str.contains(r'[^\d]+', regex=True)
non_numeric_unique_values = raw_df.loc[non_numeric_values, 'rendaMedia'].unique()
print(non_numeric_unique_values)

-----------------------------------------
Valores não numéricos na coluna rendaMedia:
['-']


In [5]:
# show rows where rendaMedia is not numeric
raw_df.loc[non_numeric_values, :]

,codigo,nome,cidade,estado,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento,potencial
232,355030170,Eta Guaraú,São Paulo,SP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,NaN,NaN
361,355030227,Pico Do Jaraguá,São Paulo,SP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,NaN,NaN
376,355030167,Reserva Da Cantareira,São Paulo,SP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-,NaN,NaN


In [6]:
# show rows where rendaMedia is null
raw_df.loc[raw_df['rendaMedia'].isnull(), :]

,codigo,nome,cidade,estado,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento,potencial
6,3304557082,Anil,Rio de Janeiro,RJ,24855,2427,1227,1777,1804,3730,5573,3556,4761,355,759,1771,1863,1902,1155,326,424,NaN,1092081.0,Baixo
26,3304557005,Catumbi,Rio de Janeiro,RJ,12910,1773,805,1018,999,1985,2526,1516,2288,0,18,261,640,1281,1140,229,703,NaN,791170.0,Baixo
52,3304557086,Freguesia (Jacarepaguá),Rio de Janeiro,RJ,72501,7991,3709,5257,5159,10923,16875,9607,12980,698,2569,6042,4941,5055,3919,704,1670,NaN,1297388.0,Médio
71,3304557146,Jacaré,Rio de Janeiro,RJ,9538,1326,629,763,712,1547,1921,1108,1532,0,15,161,492,633,792,197,540,NaN,822849.0,Baixo
89,3304557032,Maracanã,Rio de Janeiro,RJ,25969,1696,843,1370,1921,4020,5134,4124,6861,739,1695,3191,1924,1522,877,0,497,NaN,1384873.0,Baixo
122,3304557006,Rio Comprido,Rio de Janeiro,RJ,44998,5332,2397,3448,3757,7160,9375,5527,8002,0,387,2119,2775,3715,3930,617,1681,NaN,789463.0,Baixo


In [7]:
# calculo da % de dados nulos do RJ
rj_null = raw_df.loc[raw_df['estado'] == 'RJ', 'rendaMedia'].isnull().sum()
rj_total = raw_df.loc[raw_df['estado'] == 'RJ', 'rendaMedia'].shape[0]
rj_null_percent = rj_null / rj_total * 100
print('A % de dados nulos da coluna rendaMedia no estado do RJ é de {:.2f}%'.format(rj_null_percent))

A % de dados nulos da coluna rendaMedia no estado do RJ é de 3.75%


In [8]:
# drop rows where rendaMedia is null and '-' and convert to numeric
clean_df = raw_df.loc[~(raw_df['rendaMedia'].isnull() | raw_df['rendaMedia'].str.contains(r'[^\d]+', regex=True)), :].copy()
clean_df['rendaMedia'] = clean_df['rendaMedia'].astype('float64')

In [9]:
clean_df.dtypes

codigo            int64
nome             object
cidade           object
estado           object
população         int64
popAte9           int64
popDe10a14        int64
popDe15a19        int64
popDe20a24        int64
popDe25a34        int64
popDe35a49        int64
popDe50a59        int64
popMaisDe60       int64
domiciliosA1      int64
domiciliosA2      int64
domiciliosB1      int64
domiciliosB2      int64
domiciliosC1      int64
domiciliosC2      int64
domiciliosD       int64
domiciliosE       int64
rendaMedia      float64
faturamento     float64
potencial        object
dtype: object

In [10]:
clean_df.head()

,codigo,nome,cidade,estado,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento,potencial
0,3304557060,Abolição,Rio de Janeiro,RJ,11676,1027,483,688,800,1675,2300,1784,2919,0,145,715,1242,1093,758,92,304,2501.0,932515.0,Médio
1,3304557138,Acari,Rio de Janeiro,RJ,27564,5131,2188,2697,2630,4810,5308,2403,2397,0,0,82,506,2040,2490,827,2506,931.0,588833.0,Baixo
2,3304557057,Água Santa,Rio de Janeiro,RJ,9003,883,399,597,762,1755,2076,1112,1419,0,96,404,652,644,522,77,254,2391.0,874200.0,Baixo
3,3304557031,Alto Da Boa Vista,Rio de Janeiro,RJ,9606,1072,538,660,685,1317,2007,1341,1986,114,178,393,517,945,584,137,286,3727.0,912226.0,Médio
4,3304557125,Anchieta,Rio de Janeiro,RJ,57222,7677,3774,4892,4600,8660,12272,7157,8190,0,0,1089,2821,5110,5422,1073,3261,1380.0,553020.0,Médio


In [ ]:
# Verificando a quantidade de valores nulos para cada estado
clean_df.groupby('estado').apply(lambda x: x.isnull().sum())

In [ ]:
# Verificando a quantidade de valores unique para cada estado
clean_df.groupby('estado').apply(lambda x: x.nunique())

In [ ]:
# Dividindo o dataset entre raw_df_sp e raw_df_rj
clean_df_sp = clean_df[clean_df['estado'] == 'SP']
clean_df_rj = clean_df[clean_df['estado'] == 'RJ']

In [ ]:
print('-----------------------------------------')
# Valores distintos de cidade, estado e potencial para SP
cidade_sp = clean_df_sp['cidade'].unique()
bairro_sp = clean_df_sp['nome'].unique()
potencial_sp = clean_df_sp['potencial'].unique()

# Valores distintos de cidade, estado e potencial para RJ
cidade_rj = clean_df_rj['cidade'].unique()
bairro_rj = clean_df_rj['nome'].unique()
potencial_rj = clean_df_rj['potencial'].unique()

# Criando um DataFrame para armazenar os valores distintos
tabela_distintos = pd.DataFrame({
    'coluna': ['cidade','bairro', 'potencial'],
    'SP': [cidade_sp, bairro_sp, potencial_sp],
    'RJ': [cidade_rj, bairro_rj, potencial_rj]
})

# Exibindo a tabela comparativa
tabela_distintos

In [ ]:
# Analise de duplicatas em relação ao nome do bairro
print('-----------------------------------------')
print('Quantidade de duplicatas em relação ao nome do bairro:')
print('SP: ', clean_df_sp['nome'].duplicated().sum())
print('RJ: ', clean_df_rj['nome'].duplicated().sum())
# Analise de duplicatas em relação ao codigo
print('-----------------------------------------')
print('Quantidade de duplicatas em relação ao codigo do bairro:')
print('SP: ', clean_df_sp['codigo'].duplicated().sum())
print('RJ: ', clean_df_rj['codigo'].duplicated().sum())

# Novas Features

In [12]:
# clean_df_rj['popDe25a59'] = clean_df_rj['popDe25a34'] + clean_df_rj['popDe35a49']
# clean_df_rj['domiciliosA'] = clean_df_rj['domiciliosA1'] + clean_df_rj['domiciliosA2']
# clean_df_rj['domiciliosB'] = clean_df_rj['domiciliosB1'] + clean_df_rj['domiciliosB2']

# clean_df_sp['popDe25a49'] = clean_df_sp['popDe25a34'] + clean_df_sp['popDe35a49']
# clean_df_sp['domiciliosA'] = clean_df_sp['domiciliosA1'] + clean_df_sp['domiciliosA2']
# clean_df_sp['domiciliosB'] = clean_df_sp['domiciliosB1'] + clean_df_sp['domiciliosB2']

NameError: name 'clean_df_rj' is not defined